In [1]:
from src.doublethresh import double_thresh
import matplotlib.pyplot as plt
from skimage import morphology, io
from skimage import img_as_ubyte as as_ubyte

def run_analysis(hour, fov, input_dir, save_to_dir=None, 
                 lib_thresh=200, cons_thresh=160, area_threshold=100, n_dilations=3,
                 DEBUG=False):
    imgs = np.array([])
    img_paths = np.array([])
    for fname in np.sort(os.listdir(input_dir)):
        if fname.endswith('.png') and np.char.find(fname, 'trace') == -1:
            path = os.path.join(input_dir, fname)
            img_paths = np.append(img_paths, path)
            img = io.imread(path)
            if len(img.shape) > 2:
                img = img[:,:,0]
                
            if len(imgs) == 0:
                imgs = img
            else:
                imgs = np.dstack((imgs,img))
    
    if len(imgs.shape) > 2:
        img_compiled = np.amin(imgs, axis=-1)
    else:
        img_compiled = imgs

    lib_img = img_compiled < lib_thresh
    cons_img = img_compiled < cons_thresh
    
    img_double_thresh = double_thresh(cons_img, lib_img)
    
    img_dilated = np.copy(img_double_thresh)
    for i in range(n_dilations):
        img_dilated = morphology.binary_dilation(img_dilated)
    
    img_fill_holes = morphology.remove_small_holes(img_dilated, area_threshold=area_threshold)
    
    img_skel = morphology.skeletonize(img_fill_holes)

    img_pruned_skel, _, _ = pcv.morphology.prune(img_skel.astype('uint8'), 30)
    img_dilated_skel = morphology.binary_dilation(img_pruned_skel)
    
    if DEBUG:
        trace_dir = img_paths[0].split('/predict/')[0]
        trace_path = os.path.join(trace_dir, 'trace.png')

        trace = io.imread(trace_path)
        print('plotting')
        fig, axes = plt.subplots(2, 4, figsize=(15,8))

        for ax in axes.flatten():
            ax.set_axis_off()

        sub_plot(axes[0,0], img_compiled, title='Min (logical or) Combined Image')
        sub_plot(axes[0,1], img_double_thresh, title='Double Thresholded Combined Image')
        sub_plot(axes[0,2], img_dilated, title='Dilated')
        sub_plot(axes[0,3], img_fill_holes, title='Fill Holes')
        
        sub_plot(axes[1,0], img_skel, title='Double Threshold Skeleton')
        sub_plot(axes[1,1], img_pruned_skel, title='Pruned Skeleton')
        sub_plot(axes[1,2], 1 - img_dilated_skel, title='Inverted Dilated Skeleton')
        sub_plot(axes[1,3], trace, title='Hand Tracing')

        fig.tight_layout()
        plt.show()
        plt.close()

    if save_to_dir is not None:
        if not os.path.isdir(save_to_dir):
            os.mkdir(save_to_dir)
        io.imsave(os.path.join(save_dir,'1_combine.png'), img_compiled)
        io.imsave(os.path.join(save_dir,'2_thresh.png'), as_ubyte(img_double_thresh))
        io.imsave(os.path.join(save_dir,'3_dilate.png'), as_ubyte(img_dilated))
        io.imsave(os.path.join(save_dir,'4_fill.png'), as_ubyte(img_fill_holes))
        io.imsave(os.path.join(save_dir,'5_skel.png'), as_ubyte(img_skel))
        io.imsave(os.path.join(save_dir,'6_pruned_skel.png'), 255 * img_pruned_skel)
        io.imsave(os.path.join(save_dir,'7_dilated_skel.png'), as_ubyte(img_dilated_skel))

    return img_pruned_skel


# Doing Image Compilation

In [5]:
import os
from skimage import io
import numpy as np
from src.utils.process import post_process
from src.utils.sub_plot import sub_plot
from src.utils.overlay import overlay
from plantcv import plantcv as pcv
%load_ext autoreload
%autoreload 2

skel_dir = f'../data/test_all/results 03-24 (2 dilations)'
hours = ['1hr2741',
            '1hr2745',
            '1hr2753',
            '1hr2761',
            '1hr2789',
            '2hr2312',
            '2hr2314',
            '2hr2317',
            '2hr2331',
            '2hr2349',
            '2hr2354',
            '2hr2356',
            '2hr2364',
            '4hr2413',
            '4hr2414',
            '4hr2434',
            '4hr2436',
            '4hr2465',
            '4hr2495',
            '4hr2507',
            '10hr2127',
            '10hr2136',
            '10hr2144',
            '10hr2153',
            '10hr2155',
            '10hr2159',
            '10hr2400',
            '10hr2405',
            '10hr2429']

for entry in hours:
    print(entry)
    hour = entry.split('hr')[0] + 'HR'
    fov = entry.split('hr')[-1]
    fov_dir = f'../data/test_all/{hour}/{fov}'
    input_dir = os.path.join(fov_dir, 'predict')
    save_dir = os.path.join(fov_dir, 'results/03-24/')
    
    pruned_skel = run_analysis(hour, fov, input_dir, save_to_dir=save_dir, DEBUG=False, n_dilations=2)   
    if skel_dir is not None:
        io.imsave(os.path.join(skel_dir,f'{entry}.png'), 255 * pruned_skel)
    '''
    print(os.path.join(fov_dir, 'trace.png'))
    try:
        trace = io.imread(os.path.join(fov_dir, 'trace.png'))
    except FileNotFoundError:
        trace = io.imread(os.path.join(fov_dir, 'trace.tif'))
        
    if skel_dir is not None:
        io.imsave(os.path.join(skel_dir,f'{entry}_trace.png'), trace)
    '''

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1hr2741
../data/test_all/1HR/2741/trace.png
1hr2745
../data/test_all/1HR/2745/trace.png
1hr2753
../data/test_all/1HR/2753/trace.png
1hr2761
../data/test_all/1HR/2761/trace.png
1hr2789
../data/test_all/1HR/2789/trace.png
2hr2312
../data/test_all/2HR/2312/trace.png
2hr2314
../data/test_all/2HR/2314/trace.png
2hr2317
../data/test_all/2HR/2317/trace.png
2hr2331
../data/test_all/2HR/2331/trace.png
2hr2349
../data/test_all/2HR/2349/trace.png
2hr2354
../data/test_all/2HR/2354/trace.png
2hr2356
../data/test_all/2HR/2356/trace.png
2hr2364
../data/test_all/2HR/2364/trace.png
4hr2413
../data/test_all/4HR/2413/trace.png
4hr2414
../data/test_all/4HR/2414/trace.png
4hr2434
../data/test_all/4HR/2434/trace.png
4hr2436
../data/test_all/4HR/2436/trace.png
4hr2465
../data/test_all/4HR/2465/trace.png
4hr2495
../data/test_all/4HR/2495/trace.png
4hr2507
../data/test_all/4HR/2507/trace.png
10hr2127
../data/test_all/10HR/2

/var/folders/00/ljm0kfhn4fj5k8p84fdc4wd00000gn/T/ipykernel_45004/1152209762.py:61: UserWarning: ../data/test_all/results 03-24 (2 dilations)/10hr2155_trace.png is a low contrast image


10hr2159
../data/test_all/10HR/2159/trace.png
10hr2400
../data/test_all/10HR/2400/trace.png
10hr2405
../data/test_all/10HR/2405/trace.png
10hr2429
../data/test_all/10HR/2429/trace.png


In [34]:
fig, axes = plt.subplots(len(hours), 3, figsize=(13,4 * len(hours)))
    
for ax in axes.flatten():
    ax.set_axis_off()

for ind, entry in enumerate(hours):
    print(entry)
    hour = entry.split('hr')[0] + 'HR'
    fov = entry.split('hr')[-1]
    fov_dir = f'../data/test_all/{hour}/{fov}'
    
    trace_path = os.path.join(fov_dir, 'trace.png')
    trace = io.imread(trace_path)
    
    skel_path = os.path.join(skel_dir, f'{entry}.png')
    skel = io.imread(skel_path)
    
    skel_no_fill_path = os.path.join(f'../data/test_all/results 03-21', f'{entry}.png')
    skel_no_fill = io.imread(skel_no_fill_path)
    
    sub_plot(axes[ind,0], trace, title=f'{entry}\nHand Tracing', title_color='w')
    sub_plot(axes[ind,1], 255-skel, title='Skeleton With Closing', title_color='w')
    sub_plot(axes[ind,2], 255-skel_no_fill, title='Skeleton Without Closing', title_color='w')

fig.tight_layout()
print('Saving Fig')
fig.savefig('big.png', facecolor='black', bbox_inches='tight', dpi=100)
plt.close()

1hr2741
1hr2745
1hr2753
1hr2761
1hr2789
2hr2312
2hr2314
2hr2317
2hr2331
2hr2349
2hr2354
2hr2356
2hr2364
4hr2413
4hr2414
4hr2434
4hr2436
4hr2465
4hr2495
4hr2507
10hr2127
10hr2136
10hr2144
10hr2153
10hr2155
10hr2159
10hr2400
10hr2405
10hr2429
Saving Fig
